In [1]:
from amplpy import AMPL, DataFrame
import pandas as pd
import numpy as np
import os

# Load the ampl model

In [2]:
ampl = AMPL()
modelDirectory = "models"
modelName = "two_markets.mod"
mod_path = os.path.join(modelDirectory, modelName)
dataDirectory = "data"
ampl.read(mod_path)

# Assign set data

In [3]:
# Method to assign set data taking set name and a list as arguments
def assign_set_data(name,data):
    df = DataFrame(name)
    df.setColumn(name,data)
    ampl.setData(df,name)

In [4]:
intervals = pd.read_csv(os.path.join(dataDirectory, "intervals.csv"),skipinitialspace=True)
dffr = pd.read_csv(os.path.join(dataDirectory, "dffr.csv"),skipinitialspace=True)
da = pd.read_csv(os.path.join(dataDirectory, "da.csv"),skipinitialspace=True)

In [5]:
assign_set_data("INTERVALS",intervals.values[:,0])
assign_set_data("DFFR_PRICE",dffr.DFFR_PRICE.values)
assign_set_data("DA_PRICE",da.DA_PRICE.unique())

In [6]:
dffr.DFFR_PRICE.values

array([1, 2, 3, 4])

# Assign parameter data

In [14]:
ampl.getParameter('Cost').set(0)
ampl.getParameter('Ramp').set(999999)
ampl.getParameter('Ramp_DFFR').set(9999999)
ampl.getParameter('P_MAX').set(1)

In [15]:
n_DFFR_PRICE = len(dffr.DFFR_PRICE.unique())

df = pd.DataFrame([1/n_DFFR_PRICE for i in range(n_DFFR_PRICE)],
             columns=["p_R"],
             index = dffr.DFFR_PRICE.values)
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

n_DA_PRICE = len(da.DA_PRICE.unique())
df = pd.DataFrame([1/n_DA_PRICE for i in range(n_DA_PRICE)],
             columns=["p_DA"],
             index = da.DA_PRICE.unique())
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)



In [16]:
df = dffr.set_index("DFFR_PRICE")
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

df = da.set_index(["DA_PRICE","INTERVALS"])
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)


# Solve the model

In [17]:
# Set ampl options
cplexopt = 'mipdisplay = 2 mipgap = .005'
settings = {'solver': 'cplexamp', 'cplex_options': cplexopt}
for key in settings:
    ampl.setOption(key, settings[key])

In [18]:
ampl.solve()

CPLEX 12.5.1.0: mipdisplay = 2
mipgap = .005
Found incumbent of value 0.000000 after 0.00 sec. (0.02 ticks)
Dual infeasible due to empty column 'x1'.

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.34 ticks)
Parallel b&c, 32 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.34 ticks)
CPLEX 12.5.1.0: integer unbounded ray.
0 MIP simplex iterations
0 branch-and-bound nodes
No basis.


In [20]:
# Example of getting solution
p = ampl.getVariable("q_R")
p.getValues().toPandas()

,q_R.val
0,0.0


In [27]:
# Example of getting parameter values
p = ampl.getParameter("Ramp_DFFR")
p.getValues().toPandas()

,Ramp_DFFR
0,9999999.0
